In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import scipy as sy
from scipy.io import wavfile as wav #importare ed esportare file audio
import pylab
import os
import sys
import numpy as np
import pandas as pd

In [2]:
feat = pd.read_csv("D:/Data_Science_all/MSC_2_anno/PROGETTO_AML_DSIM/dataset_features.csv")

In [3]:
from sklearn.preprocessing import LabelEncoder
encoder2 = LabelEncoder()
feat["Chi2"] = encoder2.fit_transform(feat.Chi)
#feat["Chi2"] = encoder2.inverse_transform(feat.Chi)
encoder = LabelEncoder()
feat["Cosa2"] = encoder.fit_transform(feat.Cosa)

In [4]:
feat[["Chi","Chi2","Cosa","Cosa2"]].head(20)

,Chi,Chi2,Cosa,Cosa2
0,lorenzo,1,sanzione,1
1,sconosciuto,2,tipologia,2
2,sconosciuto,2,descrizione,0
3,lorenzo,1,sanzione,1
4,lorenzo,1,tipologia,2
5,giorgio,0,sanzione,1
6,giorgio,0,tipologia,2
7,sconosciuto,2,tipologia,2
8,sconosciuto,2,tipologia,2
9,lorenzo,1,tipologia,2


In [5]:
import librosa

def feat_prova(X):
    sample_rate = 44100
    # short term fourier transform: represents a signal in the time-frequency domain by computing discrete Fourier transforms (DFT) over short overlapping windows.
    stft = np.abs(librosa.stft(X))
    
    # mfcc (mel-frequency cepstrum) The mel frequency cepstral coefficients (MFCCs) of a signal are a small set of features (usually about 10-20) which concisely describe the overall shape of a spectral envelope.
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # chroma
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # melspectrogram It partitions the Hz scale into bins, and transforms each bin into a corresponding bin in the Mel Scale, using a overlapping triangular filters.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # spectral contrast Each frame of a spectrogram S is divided into sub-bands. For each sub-band, the energy contrast is estimated by comparing the mean energy in the top quantile (peak energy) to that of the bottom quantile (valley energy). High contrast values generally correspond to clear, narrow-band signals, while low contrast values correspond to broad-band noise.
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    
    return mfccs,chroma,mel,contrast,tonnetz

In [6]:
# !pip install numpy==1.17.5
# #sklearn 0.22.1
# !pip install scikit-learn==0.22.1

In [7]:
import sklearn 
sklearn.__version__

'0.22.1'

In [ ]:
!pip install

In [8]:
from joblib import dump, load
clf_chi = load('C:/Users/loren/Downloads/modello_chi_finale.joblib') 
std_scaler = load('C:/Users/loren/Downloads/std_scaler_modello_chi.bin') 
clf_cosa = load('C:/Users/loren/Downloads/modello_cosa_finale.joblib') 
std_scaler_cosa = load('C:/Users/loren/Downloads/std_scaler_modello_cosa.bin') 

In [9]:
df = pd.read_csv("D:/Data_Science_all/MSC_2_anno/PROGETTO_AML_DSIM/Object-and-Sound-detection/risposte_domande_trafficlight.csv", sep = ";")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.columns = ['title', 'target', 'descrizione', 'tipologia', 'sanzione']

In [ ]:
#!pip install sounddevice

In [ ]:
mfccs_lista,chroma_lista,mel_lista, contrast_lis

In [217]:
from gtts import gTTS
from playsound import playsound
import sounddevice as sd
import numpy as np
np.set_printoptions(suppress=True)
def question_answer(segnale, audio, string):
    #Preprocessing audio di input
    mfccs,chroma,mel,contrast,tonnetz = feat_prova(audio.reshape(110250))
    x_prova = np.concatenate([mfccs.reshape(1,40),chroma.reshape(1,12),mel.reshape(1,128),
                              contrast.reshape(1,7)],axis = 1)
    x = std_scaler.transform(x_prova)
    x_cosa = std_scaler_cosa.transform(x_prova)
    chi = ''.join(encoder2.inverse_transform(clf_chi.predict(x)))
    print(clf_chi.predict_proba(x))
    print(chi)
    cosa = ''.join(encoder.inverse_transform(clf_cosa.predict(x_cosa)))
    print(cosa)


    #risposta = df[df["title"]==segnale][cosa].iloc[0]

#     if (cosa == "descrizione"):
#         a = str("Ciao "+chi+", "+"il segnale di: "+segnale+", e' descritto nel seguente modo: "+risposta)
#     if (cosa == "sanzione"):
#         a = str("Ciao "+chi+", "+"il segnale di: "+segnale+", presenta la seguente sanzione: "+risposta)
#     if (cosa == "tipologia"):
#         a = str("Ciao "+chi+", "+"il segnale di: "+segnale+", rientra nella tipologia di: "+risposta)
    a = str("Ciao"+chi)
    language = "it"
    text = a
    myobj = gTTS(text=text, lang=language, slow=False)
    myobj.save("D:/Data_Science_all/MSC_2_anno/PROGETTO_AML_DSIM/Object-and-Sound-detection/"+str(string)+".mp3")

    return a, playsound("D:/Data_Science_all/MSC_2_anno/PROGETTO_AML_DSIM/Object-and-Sound-detection/"+str(string)+".mp3")

In [288]:
duration = 2.5 # seconds
fs = 44100
audio_z1 = sd.rec(int(duration * fs), samplerate=fs, channels=1)

In [290]:
sd.play(audio_z1)